In [ ]:
import os
import getpass

from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain_ibm import WatsonxLLM

from ibm_watsonx_ai import Credentials, APIClient
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

In [ ]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

In [5]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [6]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(credentials=credentials, project_id=project_id)

In [2]:
loader = DirectoryLoader(
    r"C:\Users\USER\Downloads\Mestrado_Dissertação\Artigos\Eligibility",
    glob="*.pdf",
    loader_cls=PyPDFLoader
)
documents = loader.load()

In [3]:
unique_articles = set(doc.metadata["source"] for doc in documents)

print(f"📄 Total articles: {len(unique_articles)}")


📄 Total articles: 170


In [ ]:
# Divide the text in chunks 
text_splitter = CharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256 
)
texts = text_splitter.split_documents(documents)

print(f"✅ Total chunks: {len(texts)}")
print(f"📄 Example:\n{texts[0].page_content[:1000]}...")


✅ Total chunks: 3287
📄 Example:
Vol.:(0123456789)1 3
Journal of Business Ethics (2020) 167:111–125 
https://doi.org/10.1007/s10551-019-04161-4
ORIGINAL PAPER
Stakeholder Relationship Capability and Firm Innovation: 
A Contingent Analysis
Wei Jiang1 · Aric Xu Wang2  · Kevin Zheng Zhou2  · Chuang Zhang3
Received: 7 September 2018 / Accepted: 11 April 2019 / Published online: 23 April 2019 
© Springer Nature B.V. 2019
Abstract
Despite the growing importance of stakeholder management, few studies have empirically examined the influence of stake-
holder relationship capability (SRC) on firm innovation, especially in emerging economies. This study investigates how SRC 
relates to firm innovation in the presence of governmental intervention and in combination with firm-level characteristics. 
Using a survey and multiple secondary datasets on the listed Chinese firms, our findings indicate that SRC is positively 
associated with firm innovation. Moreover, advanced legal development and high-te

In [10]:
api_client.foundation_models.EmbeddingModels.show()

{'GRANITE_EMBEDDING_107M_MULTILINGUAL': 'ibm/granite-embedding-107m-multilingual', 'GRANITE_EMBEDDING_278M_MULTILINGUAL': 'ibm/granite-embedding-278m-multilingual', 'SLATE_125M_ENGLISH_RTRVR': 'ibm/slate-125m-english-rtrvr', 'SLATE_125M_ENGLISH_RTRVR_V2': 'ibm/slate-125m-english-rtrvr-v2', 'SLATE_30M_ENGLISH_RTRVR': 'ibm/slate-30m-english-rtrvr', 'SLATE_30M_ENGLISH_RTRVR_V2': 'ibm/slate-30m-english-rtrvr-v2', 'MULTILINGUAL_E5_LARGE': 'intfloat/multilingual-e5-large', 'ALL_MINILM_L12_V2': 'sentence-transformers/all-minilm-l12-v2', 'ALL_MINILM_L6_V2': 'sentence-transformers/all-minilm-l6-v2'}


In [ ]:
splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
texts = splitter.split_documents(documents)

# 3. Embeddings with e5-large
embedding_function = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    encode_kwargs={"normalize_embeddings": True}
)

# 4. Index with ChromaDB
db = Chroma.from_documents(
    documents=texts,
    embedding=embedding_function,
    persist_directory="./test_db"
)

In [27]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

generate_params = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.TEMPERATURE:       0,
    GenParams.MIN_NEW_TOKENS:    1,
    GenParams.MAX_NEW_TOKENS:    1000,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]

}

In [ ]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"

In [ ]:
watsonx_llama4 = WatsonxLLM(
    model_id=model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=generate_params
)

In [ ]:
# ——————————————————————————————————————————————
# 1. Build your retriever (db must already be defined)
retriever = db.as_retriever(search_kwargs={"k": 5})

# 2. Expert system prompt
system_prompt = (
    "You are a leading expert in stakeholder management with extensive experience "
    "in complex, multi-stakeholder projects. Provide clear, strategic, and "
    "research-informed answers that address the challenges of managing diverse "
    "stakeholders across sectors, such as in collaborative initiatives involving "
    "industry, academia, and public institutions - for example, Research, Development, and Innovation projects. "
    "Maintain a professional tone and focus on practical, actionable guidance. "
)

# 3. Define a PromptTemplate that “stuffs” in the docs verbatim, but allows the model to fall back on its own knowledge
template = f"""{system_prompt}

Below is some context extracted from relevant documents. It *may* contain part of the answer.
Use it to inform your response. **If the context isn’t enough, feel free to draw on your own expertise** to give a complete, accurate answer.

**Answer in one short paragraph (3 sentences max), in prose—do not use bullet lists.**

Context:
{{context}}

Question:
{{question}}

Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# 4. Wire up a RetrievalQA chain in “stuff” mode with your custom prompt
qa = RetrievalQA.from_chain_type(
    llm=watsonx_llama4,            # your Watsonx LLM wrapper
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# ——————————————————————————————————————————————
# 5. Define your batch of stakeholder-management questions
prompts = [
    "In large collaborative projects, how can organizations overcome the challenge of aligning different stakeholders with a common vision and strategy?",
    "In collaborative R&D projects, how can organizations prevent conflicts over intellectual property rights among stakeholders?",
    "How should stakeholder conflicts and unexpected changes be managed during project execution?",
    "How can project managers monitor stakeholder satisfaction and engagement throughout the life of a collaborative project?",
    "In collaborative R&D projects involving diverse types of stakeholders, how can organizations overcome the challenge of aligning different competencies, mindsets, and goals toward a common vision?",
    "How can collaborative R&D projects manage the challenge of high stakeholder turnover and resource changes during execution?",
    "How can communication between scientific and business stakeholders be improved in collaborative innovation projects?",
    "How is stakeholder prioritization managed in collaborative R&D projects?",
    "How can teams manage the misalignment of planning horizons and working styles between universities, public institutions, and companies in collaborative R&D?",
    "In collaborative R&D projects involving stakeholders from industry and research, how can organizations handle differences in goals, language, and working pace?",
    "How can stakeholder prioritization be handled in large collaborative projects with varying levels of involvement and risk?",
    "What should be done when stakeholders fail to deliver on agreed commitments during a collaborative project?",
    "How can project teams handle the challenge of keeping industry partners engaged and aligned in high-frequency collaborative workflows?",
    "How can documentation be standardized across diverse stakeholders in large-scale projects?"
]

# 6. Loop through and ask each question

results = []
for idx, q in enumerate(prompts, 1):
    res = qa.invoke(q)

    # collect unique sources for this question
    seen = set()
    sources = []
    for doc in res["source_documents"]:
        src = doc.metadata.get("source", "unknown")
        if src not in seen:
            sources.append(src)
            seen.add(src)

    # append a clean record
    results.append({
        "Prompt_ID": idx,
        "Prompt": q,
        "Response": res["result"].strip(),
        "Sources": sources
    })

    # (optional) print output
    print("\n" + "─" * 80)
    print(f"Question {idx}:\n{q}\n")
    print("Answer:")
    print(results[-1]["Response"], "\n")
    print("Sources:")
    for s in sources:
        print("-", s)



────────────────────────────────────────────────────────────────────────────────
Question 1:
In large collaborative projects, how can organizations overcome the challenge of aligning different stakeholders with a common vision and strategy?

Answer:
Organizations can overcome the challenge of aligning different stakeholders with a common vision and strategy in large collaborative projects by formulating a clear project mission statement that sets common goals and objectives, and by maintaining good relationships with stakeholders through effective communication and engagement strategies. This involves identifying stakeholders properly and understanding their needs and expectations, as well as analyzing potential conflicts and coalitions among them. By doing so, project managers can create a positive culture change within the organization and ensure that project objectives are achieved. 

Here is the revised response in one short paragraph (3 sentences max), in prose—do not use bullet 

In [52]:
import pandas as pd
from pathlib import Path

df = pd.DataFrame(results)
file_path = "Results_llama4Maverick.xlsx"
df.to_excel(file_path, index=False)